# GA

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
from numpy.random import random
from numpy.random import randint
from tensorflow.keras import backend as K 
from tensorflow.keras.callbacks import ModelCheckpoint

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.4
Scikit-Learn 0.24.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import os

In [3]:
TRAIN_DIR = 'dataset/train'
TEST_DIR = 'dataset/test'
VALID_DIR = 'dataset/valid'
IMG_SIZE = 224

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def generators(shape, preprocessing, batch): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        zoom_range = 0.2,
        horizontal_flip = True
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        TRAIN_DIR,
        target_size = (height, width), 
        batch_size = batch,
        class_mode = 'categorical'
    )

    val_dataset = imgdatagen.flow_from_directory(
        VALID_DIR,
        target_size = (height, width), 
        batch_size = batch,
        class_mode = 'categorical'
    )
    
    test_dataset = imgdatagen.flow_from_directory(
        TEST_DIR,
        target_size = (height, width), 
        batch_size = batch,
        class_mode = 'categorical'
    )
    return train_dataset, val_dataset, test_dataset

In [5]:
#Constituicao
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19

transfer_learning = ['VGG16','VGG19','ResNet50']
sizes = [32, 64, 128, 256, 512, 1024]
batch = [16,32]
optimizers = ['RMSprop','Adagrad','Adam']
learning_rate = [0.001,0.01,0.1]

In [6]:
best_fit = 0
best_generation = 0
best_cnn = None
max_gen = 3
mutation_rate = 0.2
current_fit = 0
current_gen_cnn = []
current_gen = 1

In [7]:
import tensorflow.keras as keras
class CNN:

 def __init__(self,cnn):
    global transfer_learning
    global sizes
    global batch
    global optimizers
    global learning_rate
    
    self.chromossome = []
    self.accuracy = 0
    self.full_model = None
    self.score = None
    
    #melhor CNN quando se fez separado
    if cnn == 1:
      self.chromossome.append('VGG16')
      self.chromossome.append(3)
      self.chromossome.append([512,512,512])
      self.chromossome.append(32)
      self.chromossome.append('Adam')
      self.chromossome.append(0.001)
    
    elif cnn == 2:
      transfer_learning_x = randint(0,3)
      dense_layer = randint(1,4)
      filters = []
      for i in range(0,dense_layer):
        j = randint(0,6)
        filters.append(sizes[j])
      batch_x = randint(0,2)
      optimizer = randint(0,3)
      lr = randint(0,3)

      self.chromossome.append(transfer_learning[transfer_learning_x])
      self.chromossome.append(dense_layer)
      self.chromossome.append(filters)
      self.chromossome.append(batch[batch_x])
      self.chromossome.append(optimizers[optimizer])
      self.chromossome.append(learning_rate[lr])

    self.accuracy = 0

 def construct(self,transfer,dense,filters,batch_value,opt,learning_r):
    self.chromossome.append(transfer)        
    self.chromossome.append(dense)    
    self.chromossome.append(filters)    
    self.chromossome.append(batch_value)     
    self.chromossome.append(opt)      
    self.chromossome.append(learning_r)
    return self.chromossome

 def build(self):
    #copia do modelo pre treinado que fiz separado
    conv_model = None 
    
    if self.chromossome[0] == 'VGG16':
        vgg16 = keras.applications.vgg16
        conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
    elif self.chromossome[0] == 'VGG19':
        vgg19 = keras.applications.vgg19
        conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    elif self.chromossome[0] == 'ResNet50':
        resnet50 = keras.applications.resnet50
        conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    
    for layer in conv_model.layers:
      layer.trainable = False

    x = Flatten()(conv_model.output)

    for i in range(0,self.chromossome[1]):
      x = keras.layers.Dense(self.chromossome[2][i], activation='relu')(x)
    
    predictions = keras.layers.Dense(250, activation='softmax')(x)

    self.full_model = keras.models.Model(inputs=conv_model.inputs, outputs=predictions)
    self.full_model.compile(loss='categorical_crossentropy', optimizer=self.chromossome[4], metrics=['acc'])
    #self.full_model.compile(loss='categorical_crossentropy', optimizer=self.chromossome[4](lr=self.chromossome[5]), metrics=['acc'])

 def train(self):
    global current_fit
    
    train_dataset = None 
    val_dataset = None 
    test_dataset = None  
    
    if self.chromossome[0] == 'VGG16':
        vgg16 = keras.applications.vgg16
        train_dataset, val_dataset, test_dataset = generators((224,224), preprocessing=vgg16.preprocess_input, batch=self.chromossome[3])
    elif self.chromossome[0] == 'VGG19':
        vgg19 = keras.applications.vgg19
        train_dataset, val_dataset, test_dataset = generators((224,224), preprocessing=vgg19.preprocess_input, batch=self.chromossome[3])
    elif self.chromossome[0] == 'ResNet50':
        resnet50 = keras.applications.resnet50
        train_dataset, val_dataset, test_dataset = generators((224,224), preprocessing=resnet50.preprocess_input, batch=self.chromossome[3])
    
    mc = ModelCheckpoint('best_model_ga.h5', monitor='val_loss', mode='min', save_best_only=True)

    self.full_model.fit(train_dataset,epochs=3,validation_data=val_dataset,callbacks=[mc],workers=4,verbose=1)

    self.full_model.load_weights('best_model_ga.h5')
    self.score = self.full_model.evaluate(test_dataset)
    print('Fit: '+str(current_fit))
    print('Accuracy:', self.score[1])
    print('Chromossome:')
    print(self.chromossome)
    K.clear_session()

 def fitness(self):
    self.accuracy = (self.score[1])*100
    
 def mutate(self):
    global mutation_rate
    global transfer_learning
    global sizes
    global batch
    global optimizers
    global learning_rate
    global current_gen_cnn
    
    rand = random()
    if rand < mutation_rate:
        print('To Mutate:')
        print(self.chromossome)
        rand2 = randint(0,6)
        if rand2 == 0:
            tfl = randint(0,3)
            self.chromossome[0] = transfer_learning[tfl]
        elif rand2 == 1:
            self.chromossome[5] = self.chromossome[5] / 2
        elif rand2 == 2:
            aux = []
            for i in range(0,self.chromossome[1]):
                j = randint(0,6)
                aux.append(sizes[j])
            self.chromossome[2] = aux
        elif rand2 == 3:
            self.chromossome[3] = self.chromossome[3] / 2
        elif rand2 == 4:
            optimizer = randint(0,3)
            self.chromossome[4] = optimizers[optimizer]
        elif rand2 == 5: 
            self.chromossome[5] = self.chromossome[5] / 2
        print('After Mutate:')
        print(self.chromossome)
    
 def reproduceWith(self, parent):
   rand = randint(0,3)

   child_cnn = CNN(5)
   child_cnn.construct(self.chromossome[0],self.chromossome[1],self.chromossome[2],self.chromossome[3],self.chromossome[4],self.chromossome[5])
   print('Child:')
   print(child_cnn.chromossome)
   if rand == 0:
        child_cnn.chromossome[0] = parent.chromossome[0]
   elif rand == 1:
        child_cnn.chromossome[3] = parent.chromossome[3]
   elif rand == 2:
        child_cnn.chromossome[4] = parent.chromossome[4]    
    
   print('New CNN after Reproduce -> Child:')
   print(child_cnn.chromossome)
   return child_cnn

In [8]:
def best_fitness():
  global current_gen_cnn
  global best_fit
  global best_cnn
  
  for cnn in current_gen_cnn:
    cnn.fitness()
    if cnn.accuracy > best_fit:
      best_fit = cnn.accuracy
      best_cnn = cnn
      best_generation = current_gen

In [9]:
def mutations():
    for cnn in current_gen_cnn:
        cnn.mutate()

In [10]:
def prepare():
  global current_gen_cnn
  global current_fit

  for cnn in current_gen_cnn:
    cnn.build()
    cnn.train()
    current_fit = current_fit + 1
    
  current_fit = 0

In [11]:
def getParents():
  global current_gen_cnn
  global best_fit
  
  worst_cnn = 0
  if current_gen_cnn[0].accuracy > current_gen_cnn[1].accuracy:
   cnn_first = current_gen_cnn[0]
   cnn_second = current_gen_cnn[1]
   worst_cnn = 3
  else:
   cnn_first = current_gen_cnn[1]
   cnn_second = current_gen_cnn[0]
   worst_cnn = 3
   
  i = 0
  for cnn in current_gen_cnn:
   if (cnn.accuracy > cnn_first.accuracy) and i != 0 and i != 1:
    cnn_second = cnn_first
    cnn_first = cnn
   elif (cnn.accuracy > cnn_second.accuracy) and i != 0 and i != 1:
    cnn_second = cnn
   elif cnn.accuracy < current_gen_cnn[worst_cnn].accuracy:
    worst_cnn = i
   i = i + 1
  
  new_cnn = cnn_first.reproduceWith(cnn_second)
  current_gen_cnn[worst_cnn] = new_cnn


In [12]:
def start():
    global current_gen
    global max_gen
    global best_fit
    
    current_gen_cnn.append(CNN(2))
    current_gen_cnn.append(CNN(2))
    current_gen_cnn.append(CNN(1))
    current_gen_cnn.append(CNN(2))
    current_gen_cnn.append(CNN(2))
    
    while current_gen <= max_gen:
    
     print('STARTING GENERATION: ' + str(current_gen) )
     
     prepare()
    
     best_fitness()

     getParents()

     print('STARTING MUTATION '  )

     mutations()

     current_gen = current_gen + 1

In [13]:
start()

print('Generation: ' + str(best_generation) )
print('Chromosome:')
print(best_cnn.chromossome)
print('Test Loss:', best_cnn.score[0])
print('Test accuracy:', best_cnn.score[1])

STARTING GENERATION: 1
Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2201 steps, validate for 79 steps
Epoch 1/3
2201/2201 [==============================] - 162s 73ms/step - loss: 5.5180 - acc: 0.0084 - val_loss: 5.5230 - val_acc: 0.0040
Epoch 2/3
2201/2201 [==============================] - 160s 73ms/step - loss: 5.5069 - acc: 0.0090 - val_loss: 5.5287 - val_acc: 0.0040
Epoch 3/3
2201/2201 [==============================] - 156s 71ms/step - loss: 5.5029 - acc: 0.0096 - val_loss: 5.5301 - val_acc: 0.0048
  ...
    to  
  ['...']
79/79 [==============================] - 13s 160ms/step - loss: 5.5253 - acc: 0.0040
Fit: 0
Accuracy: 0.004
Chromossome:
['ResNet50', 1, [32], 16, 'Adam', 0.001]
Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
  ...
    to  
  ['...

Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1101 steps, validate for 40 steps
Epoch 1/3
1101/1101 [==============================] - 150s 136ms/step - loss: 5.0019 - acc: 0.1052 - val_loss: 4.4456 - val_acc: 0.1712
Epoch 2/3
1101/1101 [==============================] - 144s 130ms/step - loss: 3.5742 - acc: 0.2296 - val_loss: 3.0140 - val_acc: 0.2928
Epoch 3/3
1101/1101 [==============================] - 144s 130ms/step - loss: 3.0880 - acc: 0.2996 - val_loss: 2.7822 - val_acc: 0.3152
  ...
    to  
  ['...']
40/40 [==============================] - 12s 307ms/step - loss: 2.8449 - acc: 0.3192
Fit: 3
Accuracy: 0.3192
Chromossome:
['ResNet50', 1, [512], 32, 'Adam', 0.001]
Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
  ...
    to  
  ['...']
  ...
    to  
